In [22]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)
year = 2022

### Record selection for transactions

In [23]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sell_month DESC, name'''

sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sell_month DESC, name


In [24]:
sells_df.dtypes

name           object
sell_year       int64
sell_month      int64
buy_year        int64
buy_month       int64
sell_price    float64
buy_price     float64
diff          float64
qty             int64
sell_amt      float64
buy_amt       float64
gross         float64
pct           float64
profit        float64
kind           object
dtype: object

In [25]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).gross.sum()

sell_year  sell_month  buy_year  buy_month  name 
2022       1           2019      11         MCS     -10,500.00
                       2021      3          IVL       7,500.00
                                 6          IVL      25,500.00
                                 7          IVL      18,750.00
                                            SAT       8,250.00
                                 9          DCC       4,000.00
                                            EPG       2,200.00
                                            NER      -1,350.00
                                 10         SAT      10,500.00
                                 11         KBANK    12,192.00
                                            RCL       7,500.00
                                            SIS       3,375.00
                                            TOP       5,500.00
                                 12         IMH      26,500.00
                                            KBANK     2,496.00
     

In [26]:
sold_grp = sells_df.groupby(['name'])
sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()
sold_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_20736/2967423552.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()


,sell_amt,buy_amt,qty,gross
name,,,,
CKP,"55,000.00","50,000.00","10,000","5,000.00"
DCC,"120,000.00","116,000.00","40,000","4,000.00"
EPG,"24,600.00","22,400.00","2,000","2,200.00"
IMH,"238,800.00","202,400.00","13,000","36,400.00"
IP,"66,000.00","60,600.00","3,000","5,400.00"
IVL,"616,350.00","554,400.00","13,200","61,950.00"
KBANK,"343,200.00","328,512.00","2,400","14,688.00"
MCS,"146,500.00","167,000.00","10,000","-20,500.00"
NER,"65,700.00","67,050.00","9,000","-1,350.00"


In [27]:
sold_stocks['sell_price'] = sold_stocks['sell_amt'] / sold_stocks['qty']
sold_stocks['buy_price'] = sold_stocks['buy_amt'] / sold_stocks['qty']
cols = 'sell_amt buy_amt gross qty sell_price buy_price'.split()
sold_stocks[cols].sort_values(['name'],ascending=[True]).style.format(format_dict)

,sell_amt,buy_amt,gross,qty,sell_price,buy_price
name,,,,,,
CKP,"55,000.00","50,000.00","5,000.00","10,000",5.50,5.00
DCC,"120,000.00","116,000.00","4,000.00","40,000",3.00,2.90
EPG,"24,600.00","22,400.00","2,200.00","2,000",12.30,11.20
IMH,"238,800.00","202,400.00","36,400.00","13,000",18.37,15.57
IP,"66,000.00","60,600.00","5,400.00","3,000",22.00,20.20
IVL,"616,350.00","554,400.00","61,950.00","13,200",46.69,42.00
KBANK,"343,200.00","328,512.00","14,688.00","2,400",143.00,136.88
MCS,"146,500.00","167,000.00","-20,500.00","10,000",14.65,16.70
NER,"65,700.00","67,050.00","-1,350.00","9,000",7.30,7.45


### Record selection for active stocks

In [28]:
sql = '''
SELECT name, YEAR(buys.date) AS year, MONTH(buys.date) AS month,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE status = 'Active'
ORDER BY name, buys.date'''
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,year,month,unit_cost,qty,cost_amt
0,BCH,2021,9,21.70,"6,000","130,200.00"
1,BCH,2021,9,21.30,"6,000","127,800.00"
2,BCH,2021,12,20.50,"3,000","61,500.00"
3,BGRIM,2021,9,42.00,"3,000","126,000.00"
4,BGRIM,2021,12,39.50,"3,000","118,500.00"
5,BGRIM,2021,12,38.50,"3,000","115,500.00"
6,CKP,2021,11,5.00,"20,000","100,000.00"
7,CPNCG,2022,2,12.50,"10,000","125,000.00"
8,DIF,2019,7,14.70,"7,000","102,900.00"
9,DIF,2019,9,14.70,"5,000","73,500.00"


In [29]:
buys_df.groupby(['year','month','name']).cost_amt.sum()

year  month  name  
2016  9      MCS      167,000.00
      10     MCS      334,000.00
      11     MCS      334,000.00
2017  4      MCS      167,000.00
2018  5      JASIF    300,000.00
2019  7      DIF      102,900.00
      9      DIF       73,500.00
      11     JASIF    300,000.00
2020  2      JASIF    100,000.00
      8      DIF      117,600.00
      10     DIF      147,000.00
      11     JASIF    300,000.00
2021  2      DIF      147,000.00
      3      DIF      294,000.00
             PTTGC    226,950.00
             TISCO     96,000.00
      4      PTTGC     66,750.00
             TISCO     99,000.00
      5      NOBLE    198,000.00
      6      RATCH    135,000.00
             STA      200,000.00
      7      IVL      201,600.00
             STA       94,375.00
      8      DOHOME   200,000.00
             NOBLE     39,000.00
             TMT      163,500.00
             TU       252,000.00
      9      BCH      258,000.00
             BGRIM    126,000.00
             EPG      1

In [30]:
buys_df.groupby(['year','month']).cost_amt.sum()

year  month
2016  9         167,000.00
      10        334,000.00
      11        334,000.00
2017  4         167,000.00
2018  5         300,000.00
2019  7         102,900.00
      9          73,500.00
      11        300,000.00
2020  2         100,000.00
      8         117,600.00
      10        147,000.00
      11        300,000.00
2021  2         147,000.00
      3         616,950.00
      4         165,750.00
      5         198,000.00
      6         335,000.00
      7         295,975.00
      8         654,500.00
      9       1,465,600.00
      10        554,775.00
      11        969,000.00
      12      1,242,850.00
2022  1       1,693,500.00
      2         524,000.00
Name: cost_amt, dtype: float64

In [31]:
buys_df.cost_amt.sum()

11305900.0

In [32]:
buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_20736/3973236850.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)


,cost_amt,qty
name,,
BCH,"319,500.00","15,000"
BGRIM,"360,000.00","9,000"
CKP,"100,000.00","20,000"
CPNCG,"125,000.00","10,000"
DIF,"882,000.00","60,000"
DOHOME,"291,600.00","12,000"
EPG,"261,600.00","24,000"
GLOBAL,"291,000.00","15,000"
HREIT,"268,500.00","30,000"


In [33]:
buys_grp = buys_df.groupby(by=['name'])
dtd_stocks = buys_grp['cost_amt','qty'].sum()
dtd_stocks['avg_cost'] = dtd_stocks['cost_amt'] / dtd_stocks['qty']
dtd_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_20736/726847341.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dtd_stocks = buys_grp['cost_amt','qty'].sum()


,cost_amt,qty,avg_cost
name,,,
BCH,"319,500.00","15,000",21.30
BGRIM,"360,000.00","9,000",40.00
CKP,"100,000.00","20,000",5.00
CPNCG,"125,000.00","10,000",12.50
DIF,"882,000.00","60,000",14.70
DOHOME,"291,600.00","12,000",24.30
EPG,"261,600.00","24,000",10.90
GLOBAL,"291,000.00","15,000",19.40
HREIT,"268,500.00","30,000",8.95


In [34]:
file_name = 'unit-cost.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/unit-cost.csv',
 '\\Users\\User\\iCloudDrive\\unit-cost.csv',
 '\\Users\\User\\Dropbox\\unit-cost.csv')

In [35]:
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(csv_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(box_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(data_file)

### Extra addition

In [36]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date, name'''

sql = sql % year
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sells.date, name


In [37]:
df_sells = pd.read_sql(sql, conpf)
df_sells.head()

,name,sell_year,sell_month,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,IVL,2022,1,2022-01-05,3000,44.50,42.00,"133,204.31","126,279.08","6,925.23",5.95,895,DTD
1,DCC,2022,1,2022-01-06,40000,3.00,2.90,"119,734.21","116,256.93","3,477.28",3.45,896,DTD
2,EPG,2022,1,2022-01-06,2000,12.30,11.20,"24,545.52","22,449.62","2,095.90",9.82,897,DTD
3,IMH,2022,1,2022-01-07,1500,16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73,898,DTD
4,IMH,2022,1,2022-01-10,1500,16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73,899,DTD


In [38]:
ttl_by_month = df_sells.groupby(['sell_month'])['profit'].sum()
ttl_by_month

sell_month
1   117,432.48
2    25,076.02
Name: profit, dtype: float64

In [39]:
ttl_by_month = df_sells.groupby(['sell_year','sell_month','name'], as_index=True).agg(
    {
        'profit':['sum','count'],
    }
)
ttl_by_month

profit      
                                  sum count
sell_year sell_month name                  
2022      1          DCC     3,477.28     1
                     EPG     2,095.90     1
                     IMH    26,377.29     4
                     IVL    49,402.77     4
                     KBANK  13,200.23     3
                     MCS   -10,846.64     1
                     NER    -1,644.03     1
                     RCL     7,187.70     1
                     SAT    18,007.45     2
                     SIS     3,098.42     1
                     SYNEX   2,035.70     1
                     TOP     5,040.41     1
          2          CKP     4,767.43     1
                     IMH     9,045.50     1
                     IP      5,119.60     1
                     IVL     9,954.15     1
                     MCS   -10,347.75     1
                     TOP     6,537.09     1

In [40]:
pd.set_option('max_rows',None)
ttl_by_month.get('profit')

sum  count
sell_year sell_month name                   
2022      1          DCC     3,477.28      1
                     EPG     2,095.90      1
                     IMH    26,377.29      4
                     IVL    49,402.77      4
                     KBANK  13,200.23      3
                     MCS   -10,846.64      1
                     NER    -1,644.03      1
                     RCL     7,187.70      1
                     SAT    18,007.45      2
                     SIS     3,098.42      1
                     SYNEX   2,035.70      1
                     TOP     5,040.41      1
          2          CKP     4,767.43      1
                     IMH     9,045.50      1
                     IP      5,119.60      1
                     IVL     9,954.15      1
                     MCS   -10,347.75      1
                     TOP     6,537.09      1

In [41]:
ttl_by_month.groupby(level='name').sum()

profit      
             sum count
name                  
CKP     4,767.43     1
DCC     3,477.28     1
EPG     2,095.90     1
IMH    35,422.79     5
IP      5,119.60     1
IVL    59,356.92     5
KBANK  13,200.23     3
MCS   -21,194.39     2
NER    -1,644.03     1
RCL     7,187.70     1
SAT    18,007.45     2
SIS     3,098.42     1
SYNEX   2,035.70     1
TOP    11,577.50     2

In [42]:
ttl_by_month.sum()

profit  sum     142,508.50
        count        27.00
dtype: float64